# 使用 `nn.Block` 来定义

事实上，`nn.Sequential`是`nn.Block`的简单形式。我们先来看下如何使用`nn.Block`来实现同样的网络。

可以看到`nn.Block`的使用是通过创建一个它子类的类，其中至少包含了两个函数。

- `__init__`：创建参数。上面例子我们使用了包含了参数的`dense`层
- `forward()`：定义网络的计算

我们所创建的类的使用跟前面`net`没有太多不一样。

如何定义创建和使用`nn.Dense`比较好理解。接下来我们仔细看下`MLP`里面用的其他命令：

- `super(MLP, self).__init__(**kwargs)`：这句话调用`nn.Block`的`__init__`函数，它提供了`prefix`（指定名字）和`params`（指定模型参数）两个参数。我们会之后详细解释如何使用。

- `self.name_scope()`：调用`nn.Block`提供的`name_scope()`函数。`nn.Dense`的定义放在这个`scope`里面。它的作用是给里面的所有层和参数的名字加上前缀（prefix）使得他们在系统里面独一无二。默认自动会自动生成前缀，我们也可以在创建的时候手动指定。推荐在构建网络时，每个层至少在一个`name_scope()`里。

In [1]:
from mxnet import nd
from mxnet.gluon import nn

In [ ]:
class MLP(nn.Block):
    def __init__(self, **kwargs):
        super(MLP, self).__init__(**kwargs)
        with self.name_scope():
            #它的作用是给里面的所有层和参数的名字加上前缀（prefix）使得他们在系统里面独一无二。
            self.dense0 = nn.Dense(256)
            self.dense1 = nn.Dense(10)

    def forward(self, x):
        return self.dense1(nd.relu(self.dense0(x)))
    #x是输入数据，self.dense0(x)是输出

In [ ]:
net2 = MLP()
print(net2)
#参数初始化
net2.initialize()
#输入样本X
x = nd.random.uniform(shape=(4,20))
#输出y
y = net2(x)

## `nn.Block`到底是什么东西？

在`gluon`里，`nn.Block`是一个一般化的部件。整个神经网络可以是一个`nn.Block`，单个层也是一个`nn.Block`。我们可以（近似）无限地嵌套`nn.Block`来构建新的`nn.Block`。

`nn.Block`主要提供这个东西

1. 存储参数
2. 描述`forward`如何执行
3. 自动求导
## 那么现在可以解释`nn.Sequential`了吧

`nn.Sequential`是一个`nn.Block`容器，它通过`add`来添加`nn.Block`。它自动生成`forward()`函数，其就是把加进来的`nn.Block`逐一运行。

一个简单的实现是这样的：

In [ ]:
class Sequential(nn.Block):
    def __init__(self, **kwargs):
        super(Sequential, self).__init__(**kwargs)
    def add(self, block):
        self._children.append(block)
    def forward(self, x):
        for block in self._children:
            x = block(x)
        return x
    
net = nn.Sequential()
with net.name_scope():
    #它的作用是给里面的所有层和参数的名字加上前缀（prefix）使得他们在系统里面独一无二。
    net.add(nn.Dense(256, activation="relu")) 
    net.add(nn.Dense(10))
    
net4.initialize()
y = net4(x)##实际调用是net4.forward()

In [ ]:
class FancyMLP(nn.Block):
    def __init__(self,**kwargs):
        super(FancyMLP, self).__init__(**kwargs)
        with self.name_scope():
            self.dense = nn.Dense(256)
            #定义初始化权重
            self.weight = nd.random_uniform(shape=(256,20))
            
    def forward(self,x):
        x=nd.relu(self.dense(x))
        x=nd.relu(nd.dot(x,self.weight)+1)
        x=nd.relu(self.dense(x))
        return x
    

fancy_mlp = FancyMLP()
fancy_mlp.initialize()
y = fancy_mlp(x)

In [2]:
class RecMLP(nn.Block):
    def __init__(self, **kwargs):
        super(RecMLP, self).__init__(**kwargs)
        #`nn.Block`和`nn.Sequential`的嵌套使用
        self.net = nn.Sequential()
        with self.name_scope():
            self.net.add(nn.Dense(256, activation="relu"))
            self.net.add(nn.Dense(128, activation="relu"))
            self.dense = nn.Dense(64)

    def forward(self, x):
        return nd.relu(self.dense(self.net(x)))

rec_mlp = nn.Sequential()
rec_mlp.add(RecMLP())
rec_mlp.add(nn.Dense(10))
print(rec_mlp)

Sequential(
  (0): RecMLP(
    (net): Sequential(
      (0): Dense(None -> 256, Activation(relu))
      (1): Dense(None -> 128, Activation(relu))
    )
    (dense): Dense(None -> 64, linear)
  )
  (1): Dense(None -> 10, linear)
)


# 初始化模型参数&访问模型参数

之前我们提到过可以通过`weight`和`bias`访问`Dense`的参数，他们是`Parameter`这个类：

In [4]:
from mxnet.gluon import nn
from mxnet import nd

def get_net():
    net = nn.Sequential()
    with net.name_scope():
        net.add(nn.Dense(4, activation="relu"))
        net.add(nn.Dense(2))
    return net

x = nd.random.uniform(shape=(3,5))
net = get_net()
net.initialize()
net(x)


[[ 3.7522419e-05  1.3728756e-03]
 [-2.9933348e-03  2.0859984e-03]
 [-2.1478857e-03 -1.9505927e-03]]
<NDArray 3x2 @cpu(0)>

In [5]:
w = net[0].weight
b = net[0].bias
print('name: ', net[0].name, '\nweight: ', w, '\nbias: ', b)

name:  sequential2_dense0 
weight:  Parameter sequential2_dense0_weight (shape=(4, 5), dtype=<class 'numpy.float32'>) 
bias:  Parameter sequential2_dense0_bias (shape=(4,), dtype=<class 'numpy.float32'>)


In [6]:
print('weight:', w.data())
print('weight gradient', w.grad())
print('bias:', b.data())
print('bias gradient', b.grad())

weight: 
[[-0.0578019   0.02074406 -0.06716943 -0.01844618  0.04656678]
 [ 0.06400172  0.03894195 -0.05035089  0.0518017   0.05181222]
 [ 0.06700657 -0.00369488  0.0418822   0.0421275  -0.00539289]
 [ 0.00286685  0.03927409  0.02504314 -0.05344158  0.03088857]]
<NDArray 4x5 @cpu(0)>
weight gradient 
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
<NDArray 4x5 @cpu(0)>
bias: 
[0. 0. 0. 0.]
<NDArray 4 @cpu(0)>
bias gradient 
[0. 0. 0. 0.]
<NDArray 4 @cpu(0)>


In [8]:
params = net.collect_params() #返回的是class mxnet.gluon.ParameterDict 
print(params)
print(params['sequential2_dense0_bias'].data())
print(params.get('dense0_weight').data())

sequential2_ (
  Parameter sequential2_dense0_weight (shape=(4, 5), dtype=<class 'numpy.float32'>)
  Parameter sequential2_dense0_bias (shape=(4,), dtype=<class 'numpy.float32'>)
  Parameter sequential2_dense1_weight (shape=(2, 4), dtype=<class 'numpy.float32'>)
  Parameter sequential2_dense1_bias (shape=(2,), dtype=<class 'numpy.float32'>)
)

[0. 0. 0. 0.]
<NDArray 4 @cpu(0)>

[[-0.0578019   0.02074406 -0.06716943 -0.01844618  0.04656678]
 [ 0.06400172  0.03894195 -0.05035089  0.0518017   0.05181222]
 [ 0.06700657 -0.00369488  0.0418822   0.0421275  -0.00539289]
 [ 0.00286685  0.03927409  0.02504314 -0.05344158  0.03088857]]
<NDArray 4x5 @cpu(0)>


# 使用不同的初始函数来初始化
我们一直在使用默认的`initialize`来初始化权重（除了指定GPU `ctx`外）。它会把所有权重初始化成在`[-0.07, 0.07]`之间均匀分布的随机数。我们可以使用别的初始化方法。例如使用均值为0，方差为0.02的正态分布

In [9]:
from mxnet import init
params.initialize(init=init.Normal(sigma=0.02), force_reinit=True)#Pclass mxnet.gluon.Parameter   (class mxnet.initializer.Initializer)init
print(net[0].weight.data(), net[0].bias.data())


[[-0.01551393 -0.01576435  0.01483546 -0.02946888 -0.02146186]
 [-0.02084965 -0.0265577  -0.02949932 -0.01048284  0.02532511]
 [ 0.01790128 -0.01203189  0.02408112 -0.01942439 -0.01165124]
 [ 0.00743415  0.01860014 -0.02845151 -0.0103524   0.04017665]]
<NDArray 4x5 @cpu(0)> 
[0. 0. 0. 0.]
<NDArray 4 @cpu(0)>


In [10]:
params.initialize(init=init.One(), force_reinit=True)
print(net[0].weight.data(), net[0].bias.data())


[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
<NDArray 4x5 @cpu(0)> 
[0. 0. 0. 0.]
<NDArray 4 @cpu(0)>


In [11]:
net(x)


[[ 8.616555  8.616555]
 [12.723242 12.723242]
 [10.25157  10.25157 ]]
<NDArray 3x2 @cpu(0)>

# 共享模型参数

In [12]:
net = nn.Sequential()
with net.name_scope():
    net.add(nn.Dense(4, activation="relu"))
    net.add(nn.Dense(4, activation="relu"))
    net.add(nn.Dense(4, activation="relu", params=net[-1].params))#当前net[-1]是上一层
    net.add(nn.Dense(2))

In [13]:
net.initialize() #默认weight初始化是均匀分布继承了block的initialize？？
net(x)
print(net[0].weight.data())
print(net[1].weight.data())
print(net[2].weight.data())
print(net[3].weight.data())


[[ 0.02188614 -0.02559176 -0.05065439  0.03896836 -0.04247847]
 [ 0.06293995 -0.01837847  0.02275376  0.04493906 -0.06809997]
 [-0.05640582  0.01719845  0.04731229  0.02431235 -0.05654623]
 [ 0.06607229  0.06670433  0.05294709 -0.00438883  0.00134741]]
<NDArray 4x5 @cpu(0)>

[[ 0.06674656 -0.06219994  0.01467837 -0.00683771]
 [ 0.0334969  -0.06720173 -0.06451371 -0.00816047]
 [-0.03040703  0.06714214 -0.05317248 -0.01967777]
 [-0.02854037 -0.00267491 -0.05337812  0.02641256]]
<NDArray 4x4 @cpu(0)>

[[ 0.06674656 -0.06219994  0.01467837 -0.00683771]
 [ 0.0334969  -0.06720173 -0.06451371 -0.00816047]
 [-0.03040703  0.06714214 -0.05317248 -0.01967777]
 [-0.02854037 -0.00267491 -0.05337812  0.02641256]]
<NDArray 4x4 @cpu(0)>

[[-0.02548236  0.05326662 -0.01200318  0.05855297]
 [-0.06101935 -0.0396449   0.0269461   0.00912645]]
<NDArray 2x4 @cpu(0)>


# 自定义初始化方法
下面我们自定义一个初始化方法。它通过重载`_init_weight`来实现不同的初始化方法。（注意到Gluon里面`bias`都是默认初始化成0）

In [14]:
class MyInit(init.Initializer):#class mxnet.initializer.Initializer??重载 init=..也会重载??
    def __init__(self):
        super(MyInit, self).__init__()
        self._verbose = True
    def _init_weight(self, _, arr):
        # 初始化权重，使用out=arr后我们不需指定形状
        #print('init weight', arr.shape)
        nd.random.uniform(low=5, high=10, out=arr)
    def _init_bias(self, _, arr):
        #print('init_bias',arr.shape)
        arr[:] = 1.0
        #print('init_bias',arr.shape)

In [15]:
net = get_net()
net.initialize(MyInit())
net(x)
net[0].bias.data()


[0. 0. 0. 0.]
<NDArray 4 @cpu(0)>

1、self._verbose = True是实现什么功能？
2、“ 初始化权重，使用out=arr后我们不需指定形状”这个没整明白，这段代码是实现初始化权重参数的原理是啥呢？
3、为啥bias没有被初始化呢？（注释中的“FIXME”难道表示这个是一个没修的bug吗？）


第三点应该是bug，应该是gluon里面的parameter.py实现中忽略了bias
可以采用一种蛮无聊的强制初始化参数：
net[0].bias.initialize(init=MyInit(), force_reinit=True) （此时bias也会被赋予5~10，因此如果采用这种方式需要再写一个MyInit类）

第一点是为了“输出更全的信息”：这在后面教程中比如 y=net(x)时会输出每层的输出信息

In [ ]:
当然我们也可以通过`Parameter.set_data`来直接改写权重。注意到由于有延后初始化，所以我们通常可以通过调用一次`net(x)`来确定权重的形状先。

In [ ]:
net = get_net()
net.initialize()
net(x)

print('default weight:', net[1].weight.data())

w = net[1].weight
w.set_data(nd.ones(w.shape))

print('init to all 1s:', net[1].weight.data())